# Tracking + Anomaly Detection in Avenue Dataset

## Dataset used: **Avenue Dataset for Abnormal Event Detection**



This dataset accompanies paper "Abnormal Event Detection at 150 FPS in Matlab, Cewu Lu, Jianping Shi, Jiaya Jia, International Conference on Computer Vision, (ICCV), 2013"









`NOTE:` This is just a reference code. It is not mandatory to use the same code, make changes as u need

### Importing required libraries

In [18]:
!pip install deep-sort-realtime


In [19]:
import os
import cv2
import torch
import shutil
import random
import numpy as np
from glob import glob
from tqdm import tqdm
import matplotlib.pyplot as plt
from deep_sort_realtime.deepsort_tracker import DeepSort

np.random.seed(42)

### Mount google drive


In [20]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


```
Avenue Dataset
 └── testing_videos
 └── testing_vol
 └── training_videos
 └── training_vol
     
```

In [21]:
if not os.path.exists('yolov5'):
    !git clone https://github.com/ultralytics/yolov5.git

In [22]:
%cd yolov5/
!pwd

/content/yolov5
/content/yolov5


In [23]:
!pip install -r requirements.txt

In [24]:
%cd /content

/content


#### Use the best YOLOv5 model u have trained

In [25]:
model_path ='/content/drive/MyDrive/MOT17-RESULTS/results_12/weights/best.pt'

In [26]:
# Load YOLOv5 model
model = torch.hub.load('yolov5', 'custom', path= model_path, source='local')
model.conf = 0.25
model.iou = 0.45

YOLOv5 🚀 v7.0-457-g84ef1e59 Python-3.12.12 torch-2.9.0+cpu CPU

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


In [27]:
! pip install deep_sort_realtime opencv-python tqdm

In [28]:
output_video_folder ="/content/drive/MyDrive/anomaly_result"
os.makedirs(output_video_folder, exist_ok = True)

In [33]:
input_video_path ="/content/drive/MyDrive/testing_videos/01.avi"
output_video_path='/content/drive/MyDrive/anomaly_result/01.avi'

### Tune the below parameters

In [34]:

smoothing_frames =10   # Smoothing: once RED, stay RED for N frame

In [35]:
# Init DeepSORT tracker
tracker = DeepSort(max_age=30)

# Input video
cap = cv2.VideoCapture(input_video_path)

# Get video info
fps = cap.get(cv2.CAP_PROP_FPS)
width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print(f"Video: {fps} FPS, {width}x{height}, {frame_count} frames")

# Prepare output video writer
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

frame_idx = 0
read_count = 0
pbar = tqdm(total=frame_count)

# Store trajectory history
track_memory = {}   # {track_id: [(x, y), ...]}
N_history = 30      # number of past points to draw

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # YOLO inference
    results = model(frame)
    detections = results.xyxy[0].cpu().numpy()  # (xmin, ymin, xmax, ymax, conf, cls)

    # Format detections for DeepSORT
    formatted_detections = []
    for d in detections:
        xmin, ymin, xmax, ymax, conf, cls = d
        box = [xmin, ymin, xmax - xmin, ymax - ymin]
        formatted_detections.append([box, conf])

    # Update tracker
    tracks = tracker.update_tracks(formatted_detections, frame=frame)

    # Draw tracks
    for track in tracks:
        if not track.is_confirmed():
            continue

        track_id = track.track_id
        xmin, ymin, xmax, ymax = map(int, track.to_ltrb())

        # Center point
        cx = int((xmin + xmax) / 2)
        cy = int((ymin + ymax) / 2)

        # Update history
        track_memory.setdefault(track_id, []).append((cx, cy))
        track_memory[track_id] = track_memory[track_id][-N_history:]

        # Draw bounding box
        color = (0, 255, 0)
        label = f"ID {track_id}"

        cv2.rectangle(frame, (xmin, ymin), (xmax, ymax), color, 2)
        cv2.putText(frame, label, (xmin, ymin - 8),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

        # Draw trajectory (trace)
        for i in range(1, len(track_memory[track_id])):
            cv2.line(
                frame,
                track_memory[track_id][i - 1],
                track_memory[track_id][i],
                color,
                2
            )

    out.write(frame)

    frame_idx += 1
    read_count += 1
    pbar.update(1)

pbar.close()
cap.release()
out.release()

print(f"Frames read: {read_count} / {frame_count}")
print(f"\nDONE — output saved to: {output_video_path}")


Video: 25.0 FPS, 640x360, 1439 frames


100%|██████████| 1439/1439 [14:18<00:00,  1.68it/s]

Frames read: 1439 / 1439

DONE — output saved to: /content/drive/MyDrive/anomaly_result/01.avi


In [32]:
import os
print(os.listdir("/content/drive/MyDrive/"))


['Colab Notebooks', 'MOT17', 'MOT17-FRCNN-YOLO', 'MOT17-RESULTS', 'testing_videos', 'anomaly_result']




####  Check for different test videos